In [1]:
import copy
import math
import os

#Pymatgen
from pymatgen.analysis.defects.generators import VacancyGenerator
from pymatgen.ext.matproj import MPRester
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.structure_analyzer import RelaxationAnalyzer
import importlib

from ase.io import read as ase_read
from ase.io.cube import write_cube
import numpy as np

/var/folders/wk/ppfd8h190c71vfpctvycy4zc0000gn/T/ipykernel_58944/2737101172.py:6: DeprecationWarning: The pymatgen.analysis.defects module will be deprecated and replaced by a modified name-space package in the near future.
  from pymatgen.analysis.defects.generators import VacancyGenerator


In [9]:
# Sometimes the ASE code for fetching relax calculations does not return the truly last set of coordinates.
# This code will parse a Quantum Espresso *.pwo file and get the coordinates
# Only works for 'relax' calculations
# Parameter | filename - filename of *.pwo 'relax' calculation
def force_fetch_final_coordinates(filename):

    coordinates_found = False
    all_coordinates = []

    line_info_begin = 'Begin final coordinates'
    line_info_end = 'End final coordinates'

    line_start = -1
    line_end = -1
    line_count = 1

    try:
        with open(filename) as file:
            for line in file:
                if line_info_begin in line:
                    line_start = line_count + 3
                if line_info_end in line:
                    line_end = line_count
                line_count += 1
    except:
            print("File not found! \n")

    if(line_start > 0 and line_end > 0):
        coordinates_found = True

    line_count = 1

    if(coordinates_found):
        with open(filename) as file:
            for line in file:
                if (line_count >= line_start and line_count < line_end):
                    line_info = line.split()
                    x_coord = float(line_info[1])
                    y_coord = float(line_info[2])
                    z_coord = float(line_info[3])
                    single_coord = [x_coord,y_coord,z_coord]
                    all_coordinates.append(single_coord)
                line_count += 1

    return coordinates_found,all_coordinates


In [28]:
#Calculates the distance atoms have shifted between the start and end files
# Parameter | supercell_filename_start - Starting location
# Parameter | supercell_filename_end - Ending Location
# Parameter | skip_start - Select the atoms to not consider in the start file
# Parameter | skip_end - Select the atoms to not consider in the end file
# Parameter | defect_index_from_start - Select which atom in the start file represents the defect location
# Parameter | shortest_bond_length - Shortest bond length of crystal system
# Parameter | shortest_bond_multiplier - Defines the radius where relaxation is considered
#                                        where (r = shortest_bond_length*shortest_bond_multiplier)
# Parameter | use_manual_final_locs - If true, allow manual fetching. If false, will only use
#                                     ASE fetching
# Parameter | account_for_new_atom - If true, consider how a new antisite atom moves w.r.t its corresponding
#                                    pristine atom location
# Parameter | save_cubes - If true, will create a cube file with the pristine and relaxed atoms (in relaxation radius)
def calculate_relaxation_within_radius_allow_defect_include(supercell_filename_start,
                                                            supercell_filename_end,
                                                            skip_start,
                                                            skip_end,
                                                            defect_index_from_start,
                                                            shortest_bond_length,
                                                            shortest_bond_multiplier,
                                                            use_manual_final_locs = True,
                                                            account_for_new_atom = False,
                                                            save_cubes = False):

    calculation_warning = False

    #define the starting supercell
    first_atom_loc = ase_read(supercell_filename_start)

    #Check the sites manually
    start_manual_result, start_manual_site_locs = force_fetch_final_coordinates(supercell_filename_start)
    if(start_manual_result == True):
        if((start_manual_site_locs == first_atom_loc.get_positions()).all()):
            print("The manual and ASE methods return the SAME start locations")
        else:
            print("The manual and ASE methods return DIFFERENT start locations")
        if(use_manual_final_locs):
            first_atom_loc.set_positions(start_manual_site_locs)

    first_loc_pymatgen_struct = AseAtomsAdaptor.get_structure(atoms = first_atom_loc)

    #get the original defects location
    defect_loc = first_loc_pymatgen_struct.sites[defect_index_from_start].coords
    defect_loc = copy.deepcopy(defect_loc)

    #now remove the undesired atoms
    if(account_for_new_atom == True):
        atom_at_unrelaxed_defect_loc = first_loc_pymatgen_struct.sites[defect_index_from_start]
        first_loc_pymatgen_struct.remove_sites(skip_start)
        first_loc_pymatgen_struct.sites.append(atom_at_unrelaxed_defect_loc)
    else:
        first_loc_pymatgen_struct.remove_sites(skip_start)

    #define the final supercell
    last_atom_loc = ase_read(supercell_filename_end)

    #Check the sites manually
    manual_result, manual_site_locs = force_fetch_final_coordinates(supercell_filename_end)
    if(manual_result == True):
        if((manual_site_locs == last_atom_loc.get_positions()).all()):
            print("The manual and ASE methods return the SAME final locations")
        else:
            print("The manual and ASE methods return DIFFERENT final locations")
        if(use_manual_final_locs):
            last_atom_loc.set_positions(manual_site_locs)

    #Convert last loc to pymatgen structure
    last_loc_pymatgen_struct = AseAtomsAdaptor.get_structure(atoms = last_atom_loc)

    if(account_for_new_atom == False):
        last_loc_pymatgen_struct.remove_sites(skip_end)

    #FYI: if correctly defined, removing skip_start and skip_end will result in the matching
    # starting and ending atoms having the SAME index
    # You can test this by putting in funky numbers, the amount of movement will be huge

    #Make sure they have the same number of atoms
    atoms_in_start = first_loc_pymatgen_struct.num_sites
    atoms_in_end = last_loc_pymatgen_struct.num_sites

    if(atoms_in_start != atoms_in_end):
        print(f"The number of atoms in the start and end do not match.")
        return [False,False,True]

    #Define supercell
    lattice_a = first_loc_pymatgen_struct.lattice.a
    lattice_b = first_loc_pymatgen_struct.lattice.b
    lattice_c = first_loc_pymatgen_struct.lattice.c

    #If the radius is larger, it will start to include atoms near to image defects
    max_reasonable_relax_distance = min(lattice_a,lattice_b,lattice_c)/2
    print(f"The max reasonable relax radius: {max_reasonable_relax_distance}")

    defect_radius_max = shortest_bond_length*shortest_bond_multiplier
    print(f"Defect Max Radius: {defect_radius_max}")

    supercell_mult_a = math.ceil(defect_radius_max/lattice_a)
    supercell_mult_b = math.ceil(defect_radius_max/lattice_b)
    supercell_mult_c = math.ceil(defect_radius_max/lattice_c)

    supercell_dim_a = 2*supercell_mult_a + 1
    supercell_dim_b = 2*supercell_mult_b + 1
    supercell_dim_c = 2*supercell_mult_c + 1

    """
    The supercell is created such that the defect of interest is placed at the center
    of the new supercell. The supercell size is such that it can fully accomodate the
    defect_radius_max.
    """

    #Update the defect location to be at the center of the new supercell
    defect_loc += (first_loc_pymatgen_struct.lattice.matrix[0]*supercell_mult_a)
    defect_loc += (first_loc_pymatgen_struct.lattice.matrix[1]*supercell_mult_b)
    defect_loc += (first_loc_pymatgen_struct.lattice.matrix[2]*supercell_mult_c)
    #print(f"New defect loc: {defect_loc}")

    #Now apply the supercell to the start and final locations
    first_loc_pymatgen_struct.make_supercell([supercell_dim_a,supercell_dim_b,supercell_dim_c])
    last_loc_pymatgen_struct.make_supercell([supercell_dim_a,supercell_dim_b,supercell_dim_c])

    #Apply supercell to demostration cube file
    atoms_relaxed_pristine_pymatgen = copy.deepcopy(first_loc_pymatgen_struct)
    atoms_relaxed_defect_pymatgen = copy.deepcopy(last_loc_pymatgen_struct)

    dist_sum = 0
    atoms_included = 0

    atoms_in_radius = []
    dist_from_start = []

    for index in range(0,first_loc_pymatgen_struct.num_sites):

        first_loc = first_loc_pymatgen_struct.sites[index].coords
        last_loc = last_loc_pymatgen_struct.sites[index].coords
        dist = np.linalg.norm(first_loc - last_loc)

        #Check if within defect radius
        pristine_atom_distance_from_defect_site = np.linalg.norm(first_loc - defect_loc)

        #Defines sphere of relaxation w.r.t. the pristine lattice structure
        if pristine_atom_distance_from_defect_site < defect_radius_max:

            atoms_included += 1
            atoms_in_radius.append(index)

            if(dist > 1):
                print(f"Warning! Atom {index} moved a large amount")
                print(f"Atom {index} first loc: {first_loc_pymatgen_struct.sites[index]}")
                print(f"Atom {index} last loc: {last_loc_pymatgen_struct.sites[index]}")
                print(f"")
                calculation_warning = True

            dist_sum += dist
            dist_from_start.append(dist)
        else:
            # Save atoms outside radius as another atom type
            # Makes it easy to filter out in cube file, but doesn't mess up indexing
            atoms_relaxed_pristine_pymatgen.sites[index].species = 'Mt1'
            atoms_relaxed_defect_pymatgen.sites[index].species = 'Mt1'


    avg_relaxation = dist_sum/atoms_included
    normed_relaxation = avg_relaxation/shortest_bond_length

    #print(f"Atom Indeces: {atoms_in_radius}")
    #print(f"Total Movement: {dist_sum} angstroms")
    #print(f"Number of Atoms in calculation: {atoms_included} atoms")

    print(f"Average Relaxation: {avg_relaxation} angstroms/atom")
    print(f"Normalized Relaxation: {normed_relaxation} /atom")

    if(save_cubes):
        atoms_relaxed_pristine = AseAtomsAdaptor.get_atoms(atoms_relaxed_pristine_pymatgen)
        atoms_relaxed_defect = AseAtomsAdaptor.get_atoms(atoms_relaxed_defect_pymatgen)

        cube_name = f"/Users/andrewtimmins/Desktop/relaxed_pristine.cube"

        with open(cube_name,'w') as file:
            write_cube(file, atoms=atoms_relaxed_pristine)

        cube_name = f"/Users/andrewtimmins/Desktop/relaxed_defect.cube"

        with open(cube_name,'w') as file:
            write_cube(file, atoms=atoms_relaxed_defect)

    return [avg_relaxation,normed_relaxation,calculation_warning]


In [29]:
#Looks at a folder and finds the highest indexed relaxation file
def find_max_relax_file(folder_name):

    def checkIfFileExists(filename):
        file_found = False

        item_list = os.listdir()
        for item in item_list:
            if filename in item:
                file_found = True

        return file_found

    os.chdir(folder_name)

    #Naive way to create ordering of files. But it works.
    relax_filenames = ["espresso_relax15.pwo",
                       "espresso_relax14.pwo",
                       "espresso_relax13.pwo",
                       "espresso_relax12.pwo",
                       "espresso_relax11.pwo",
                       "espresso_relax10.pwo",
                       "espresso_relax9.pwo",
                       "espresso_relax8.pwo",
                       "espresso_relax7.pwo",
                       "espresso_relax6.pwo",
                       "espresso_relax5.pwo",
                       "espresso_relax4.pwo",
                       "espresso_relax3.pwo",
                       "espresso_relax2.pwo",
                       "espresso_relax1.pwo",
                       "espresso_relax.pwo"]

    accepted_relax_filename = ""
    accepted_relax_found = False

    for relax_filename in relax_filenames:
        relax_found = checkIfFileExists(filename=relax_filename)
        if(relax_found):
            accepted_relax_filename = relax_filename
            accepted_relax_found = True
            break

    full_path = f"{folder_name}/{accepted_relax_filename}"

    return accepted_relax_found,full_path

In [35]:
skip_start = [18]
skip_end = []
defect_index_from_start = skip_start[0]
shortest_bond_length = 3.30593 #pbI2 - 3.30593, GaN - 1.98352, C - 1.54687
shortest_bond_multiplier = 2 #Use 2 for all crystal systems
use_manual_final_locs = True
account_for_new_atom = True
save_cubes = False

supercell_folder = "/Users/andrewtimmins/Documents/CMU/Research/QE_ASE/dsProject/Testing/BEEF_Research/PbI2_mp22893/V_I/experiments/ex59_332/"
type = "VAC"
charges = [1,0,-1]


for q in charges:
    supercell_folder_start = f"{supercell_folder}/supercells/PRISTINE_0/0"
    supercell_folder_end = f"{supercell_folder}/supercells/{type}_1/{q}/"

    start_result,supercell_filename_start = find_max_relax_file(supercell_folder_start)
    end_result,supercell_filename_end = find_max_relax_file(supercell_folder_end)

    print(f"supercell_filename_start: {supercell_filename_start}")
    print(f"supercell_filename_end: {supercell_filename_end}")

    valid_files_found = False
    if(start_result == True or end_result == True):
        valid_files_found = True

    if(valid_files_found):

        account_for_new_atom = False
        standard_relax_result = calculate_relaxation_within_radius_allow_defect_include(supercell_filename_start,
                                                                                        supercell_filename_end,
                                                                                        skip_start,
                                                                                        skip_end,
                                                                                        defect_index_from_start,
                                                                                        shortest_bond_length,
                                                                                        shortest_bond_multiplier,
                                                                                        use_manual_final_locs,
                                                                                        account_for_new_atom,
                                                                                        save_cubes)

        account_for_new_atom = True
        all_relax_result = calculate_relaxation_within_radius_allow_defect_include(supercell_filename_start,
                                                                supercell_filename_end,
                                                                skip_start,
                                                                skip_end,
                                                                defect_index_from_start,
                                                                shortest_bond_length,
                                                                shortest_bond_multiplier,
                                                                use_manual_final_locs,
                                                                account_for_new_atom,
                                                                save_cubes)

        print(f"#########################################################")
        print(f"Charge: {q}")
        print(f"Relaxation WITHOUT Defect Atoms")
        print(f"   Calculation Produced Warning: {standard_relax_result[2]}")
        print(f"   Average Relaxation: {standard_relax_result[0]} [Angstrom/Atom]")
        print(f"   Normed Relaxation: {standard_relax_result[1]} [/Atom]")
        print(f"Relaxation WITH Defect Atoms")
        try:
            print(f"   Calculation Produced Warning: {standard_relax_result[2]}")
            print(f"   Average Relaxation: {standard_relax_result[0]} [Angstrom/Atom]")
            print(f"   Normed Relaxation: {standard_relax_result[1]} [/Atom]")
        except:
            print("   Relaxation w/ Defect Atoms: Not Available")
        print(f"#########################################################")

supercell_filename_start: /Users/andrewtimmins/Documents/CMU/Research/QE_ASE/dsProject/Testing/BEEF_Research/PbI2_mp22893/V_I/experiments/ex59_332//supercells/PRISTINE_0/0/espresso_relax2.pwo
supercell_filename_end: /Users/andrewtimmins/Documents/CMU/Research/QE_ASE/dsProject/Testing/BEEF_Research/PbI2_mp22893/V_I/experiments/ex59_332//supercells/VAC_1/1//espresso_relax1.pwo
The manual and ASE methods return DIFFERENT start locations
The manual and ASE methods return the SAME final locations
The max reasonable relax radius: 7.066901172464172
Defect Max Radius: 6.61186
Average Relaxation: 0.03946751813352873 angstroms/atom
Normalized Relaxation: 0.011938401034967083 /atom
The manual and ASE methods return DIFFERENT start locations
The manual and ASE methods return the SAME final locations
The number of atoms in the start and end do not match.
#########################################################
Charge: 1
Relaxation WITHOUT Defect Atoms
   Calculation Produced Warning: False
   Aver